# Feature Engineering para Precios Inmobiliarios Dinamarca

**Objetivo**: Transformación y construcción de variables predictivas para el modelado supervisado de precios de vivienda en Dinamarca.

**Contenido**:
1. Carga de datos y configuración inicial
2. Pipeline de feature engineering modular
3. Enriquecimiento geográfico
4. Documentación de resultados

In [2]:
# Configuración y carga de módulos
import pandas as pd
import warnings
from pathlib import Path
import json
from datetime import datetime
import sys

import setup
setup.set_project_root()

from config import *
from feature_engineering import enhanced_feature_engineering_pipeline
from descriptive_analysis import load_and_validate_data

warnings.filterwarnings('ignore')
print("Configuración cargada.")

Current working directory: /home/exodia/Documentos/TFBigData
Configuración cargada.
Configuración cargada.


In [3]:
# Carga de datos
df = pd.read_parquet(CLEAN_FILE)
df['date'] = pd.to_datetime(df['date'])

print(f"Datos cargados: {df.shape[0]:,} filas, {df.shape[1]} columnas")
print(f"Rango temporal: {df['date'].min().date()} a {df['date'].max().date()}")

Datos cargados: 1,506,591 filas, 19 columnas
Rango temporal: 1992-01-05 a 2024-09-30


## Pipeline de Feature Engineering con Enriquecimiento Geográfico

In [ ]:
# Pipeline completo (sin enriquecimiento geográfico temporalmente)
print("Ejecutando pipeline de feature engineering...")

output_dir = DATA_DIR / "processed"

# Usar el pipeline básico primero
from feature_engineering import run_complete_feature_engineering_pipeline

results = run_complete_feature_engineering_pipeline(
    df=df,
    target_col=TARGET,
    output_dir=output_dir
)

# Extraer resultados
df_final = results['final_dataset']
selected_features = results['selected_features']
metadata = results['metadata']
saved_files = results['saved_files']

print(f"\nPipeline completado.")
print(f"Dataset final: {df_final.shape[0]:,} filas x {df_final.shape[1]} columnas")
print(f"Features seleccionadas: {len(selected_features)}")

# Agregar enriquecimiento geográfico simple aquí directamente
if 'region' in df_final.columns:
    print("Agregando características geográficas...")
    
    # Mapeo simple de densidad urbana por región
    urban_density_map = {
        'Copenhagen': 5, 'Aarhus': 4, 'Odense': 3, 'Aalborg': 3,
        'Frederiksberg': 5, 'Esbjerg': 2, 'Randers': 2, 'Kolding': 2
    }
    
    df_final['urban_density'] = df_final['region'].map(urban_density_map).fillna(1)
    df_final['location_type'] = df_final['urban_density'].apply(
        lambda x: 'Urban' if x >= 4 else 'Suburban' if x >= 2 else 'Rural'
    )
    
    print("Características geográficas agregadas: urban_density, location_type")

print(f"Archivos guardados en: {output_dir}")

Ejecutando pipeline de feature engineering...


In [ ]:

results = run_complete_feature_engineering_pipeline(
    df=df,
    target_col=TARGET,
    output_dir=output_dir
)

# Extraer resultados
df_final = results['final_dataset']
selected_features = results['selected_features']
metadata = results['metadata']
saved_files = results['saved_files']

print(f"\nPipeline completado.")
print(f"Dataset final: {df_final.shape[0]:,} filas x {df_final.shape[1]} columnas")
print(f"Features seleccionadas: {len(selected_features)}")

# Agregar enriquecimiento geográfico simple aquí directamente
if 'region' in df_final.columns:
    print("Agregando características geográficas...")
    
    # Mapeo simple de densidad urbana por región
    urban_density_map = {
        'Copenhagen': 5, 'Aarhus': 4, 'Odense': 3, 'Aalborg': 3,
        'Frederiksberg': 5, 'Esbjerg': 2, 'Randers': 2, 'Kolding': 2
    }
    
    df_final['urban_density'] = df_final['region'].map(urban_density_map).fillna(1)
    df_final['location_type'] = df_final['urban_density'].apply(
        lambda x: 'Urban' if x >= 4 else 'Suburban' if x >= 2 else 'Rural'
    )
    
    print("Características geográficas agregadas: urban_density, location_type")

print(f"Archivos guardados en: {output_dir}")

## Documentación de Artefactos

In [5]:
# Documentación de archivos generados
print(f"{'Archivo':<45} | {'Descripción'}")
print("-" * 80)

file_descriptions = {
    "feature_engineered_complete.parquet": "Dataset con todas las features generadas",
    "modeling_dataset.parquet": "Dataset final para modelado",
    "train_data.parquet": "Conjunto de entrenamiento",
    "test_data.parquet": "Conjunto de prueba",
    "selected_features.txt": "Lista de features seleccionadas",
    "scalers.pkl": "Escaladores ajustados",
    "feature_engineering_metadata.json": "Metadatos del proceso",
    "feature_engineered_with_geography.parquet": "Dataset con enriquecimiento geográfico"
}

for name, path_obj in saved_files.items():
    if hasattr(path_obj, 'exists') and path_obj.exists():
        desc = file_descriptions.get(path_obj.name, "Archivo auxiliar")
        print(f"{path_obj.name:<45} | {desc}")

# Guardar documentación final
doc_path = output_dir / "feature_engineering_summary.json"
summary = {
    "fecha": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "dataset_shape": f"{df_final.shape[0]} x {df_final.shape[1]}",
    "features_count": len(selected_features),
    "geographic_enrichment": 'geographic_features' in results,
    "archivos_generados": {name: str(path) for name, path in saved_files.items()}
}

with open(doc_path, 'w') as f:
    json.dump(summary, f, indent=2)

print(f"\nDocumentación guardada: {doc_path.name}")

Archivo                                       | Descripción
--------------------------------------------------------------------------------
feature_engineered_complete.parquet           | Dataset con todas las features generadas
modeling_dataset.parquet                      | Dataset final para modelado
train_data.parquet                            | Conjunto de entrenamiento
test_data.parquet                             | Conjunto de prueba
scalers.pkl                                   | Escaladores ajustados
selected_features.txt                         | Lista de features seleccionadas
feature_engineering_metadata.json             | Metadatos del proceso
feature_engineering_summary.md                | Archivo auxiliar

Documentación guardada: feature_engineering_summary.json


## Proceso Completado

El pipeline de feature engineering ha sido ejecutado exitosamente con enriquecimiento geográfico incluido. Los datos están listos para el modelado supervisado.